In [1]:
!pip install -Uq langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00


# Document Splitter

In [2]:
text_data = """
Text mining, text data mining (TDM) or text analytics is the process of deriving high-quality information from text.
It involves "the discovery by computer of new, previously unknown information, by automatically extracting information from different written resources."
[1] Written resources may include websites, books, emails, reviews, and articles.
High-quality information is typically obtained by devising patterns and trends by means such as statistical pattern learning.
According to Hotho et al. (2005) we can distinguish between three different perspectives of text mining: information extraction, data mining,
and a knowledge discovery in databases (KDD) process.
[2] Text mining usually involves the process of structuring the input text (usually parsing,
along with the addition of some derived linguistic features and the removal of others,
and subsequent insertion into a database), deriving patterns within the structured data, and finally evaluation and interpretation of the output.
'High quality' in text mining usually refers to some combination of relevance, novelty, and interest.
Typical text mining tasks include text categorization, text clustering, concept/entity extraction,
production of granular taxonomies, sentiment analysis, document summarization, and entity relation modeling (i.e., learning relations between named entities).
"""

In [11]:
!pip install -Uq requests

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
from typing import List
import requests

In [16]:
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

In [4]:
chunk_size = 300
chunk_overlap = 30
separators: list[str] = ['n\n', '\n', ' ', '']

In [6]:
char_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=separators,
    length_function=len,
    is_separator_regex=False
)

In [7]:
text_output = char_text_splitter.split_text(text_data)

In [8]:
text_output

['Text mining, text data mining (TDM) or text analytics is the process of deriving high-quality information from text. \nIt involves "the discovery by computer of new, previously unknown information, by automatically extracting information from different written resources."',
 '[1] Written resources may include websites, books, emails, reviews, and articles. \nHigh-quality information is typically obtained by devising patterns and trends by means such as statistical pattern learning.',
 'According to Hotho et al. (2005) we can distinguish between three different perspectives of text mining: information extraction, data mining, \nand a knowledge discovery in databases (KDD) process.\n[2] Text mining usually involves the process of structuring the input text (usually parsing,',
 'along with the addition of some derived linguistic features and the removal of others, \nand subsequent insertion into a database), deriving patterns within the structured data, and finally evaluation and interp

# Text Embedding

In [17]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [43]:
'''embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)'''
embeddings = HuggingFaceEmbeddings()

<ipython-input-43-68b1626ae145>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [20]:
vector_embeddings = embeddings.embed_query(text_data)

In [22]:
len(vector_embeddings)

768

# Vector Database

In [2]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 14.9 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import PyPDFLoader

In [18]:
pdf_url = 'https://arxiv.org/pdf/2312.16862.pdf'
pdf_loader = PyPDFLoader('/content/Alice Clark CV.pdf')
pdf_pages = pdf_loader.load()

In [19]:
len(pdf_pages)

2

In [20]:
type(pdf_pages)

list

In [21]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [22]:
chunk_size = 50
chunk_overlap = 20

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False
)

In [23]:
docs = splitter.split_documents(pdf_pages)

In [24]:
type(docs[0])

langchain_core.documents.base.Document

In [25]:
docs

[Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='Alice Clark \nAI / Machine Learning'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='Delhi, India Email me on Indeed'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='• 20+ years of experience in data handling,'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='in data handling, design, and development'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='• Data Warehouse: Data analysis, star/snow flake'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='star/snow flake scema data modelling and design'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='and design specific to'),
 Document(metadata={'source': '/content/Alice Clark CV.pdf', 'page': 0}, page_content='data ware

In [26]:
!pip install -Uq langchain-chroma

In [27]:
from langchain_chroma import Chroma

In [28]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [29]:
# Use Chroma.from_documents with plain texts
chroma_db = Chroma.from_documents(docs, embedding=embedding_model)


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [30]:
query = 'collect all skills'
similar_docs = chroma_db.similarity_search(query, k=4)

In [31]:
similar_docs

[Document(metadata={'page': 0, 'source': '/content/Alice Clark CV.pdf'}, page_content='2001 \n \nSKILLS'),
 Document(metadata={'page': 1, 'source': '/content/Alice Clark CV.pdf'}, page_content='skills with ability to interact with individuals'),
 Document(metadata={'page': 1, 'source': '/content/Alice Clark CV.pdf'}, page_content='• Quick learner and maintains cordial'),
 Document(metadata={'page': 0, 'source': '/content/Alice Clark CV.pdf'}, page_content='performing rewards')]